<a href="https://colab.research.google.com/github/uguazelli/Blockchain-Multisig/blob/main/sequential_chaining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Updated version to work with LangChain 0.1.x+ and langchain-openai

# Sequential Chaining (Multi-Step Prompting)
Use multi-step chaining if:
* You need very reliable results
* You want to inject logic or validation between steps
* You're building an agent or a dynamic flow

# Pip Install

In [ ]:
# Install required packages
!pip install -U langchain langchain-community langchain-openai
!pip install pydantic


# Imports

In [6]:
# Imports'
from google.colab import userdata
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI # ✅ use langchain_openai, not community
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
import json

In [7]:
# Get API key from Colab secrets
api_key = userdata.get('OPENAI_API_KEY')

In [8]:
# Define LLM (completion model like GPT-3.5-turbo-instruct and level of creativity 0.6 of 1)
llm = OpenAI(model="gpt-3.5-turbo-instruct", api_key=api_key, temperature=0.6)

In [ ]:
# Test simple call
response = llm.invoke("Tell me a fun fact about AI")
print("Simple Response:", response)

In [ ]:
# Step 1: Name
name_prompt = PromptTemplate.from_template(
    "I want to open a {topic} restaurant. Suggest a really nice name. Return just one name."
)

# Step 2: Slogan
slogan_prompt = PromptTemplate.from_template(
    "Create a catchy slogan for a restaurant named '{name}'."
)

# Step 3: Concept
concept_prompt = PromptTemplate.from_template(
    "Describe the branding and interior design concept for a restaurant named '{name}' with the slogan '{slogan}'."
)

# Step 4: Logo Description
logo_prompt = PromptTemplate.from_template(
    "Based on the name '{name}' and slogan '{slogan}', describe an ideal logo: colors, icon, font, and style."
)

# Step 5: Image Generation Prompt (for DALL·E or Midjourney)
image_prompt = PromptTemplate.from_template(
    "Create a visual prompt for an image generation tool to design the logo: '{logo_description}'"
)

In [ ]:
# Build each step
step1 = name_prompt | llm
step2 = lambda name: slogan_prompt | llm
step3 = lambda name, slogan: concept_prompt | llm
step4 = lambda name, slogan: logo_prompt | llm
step5 = lambda logo_description: image_prompt | llm

In [ ]:
# Wrap in RunnableSequence
def full_pipeline(topic: str):
    name = step1.invoke({"topic": topic}).strip()
    slogan = (slogan_prompt | llm).invoke({"name": name}).strip()
    concept = (concept_prompt | llm).invoke({"name": name, "slogan": slogan}).strip()
    logo = (logo_prompt | llm).invoke({"name": name, "slogan": slogan}).strip()
    image_prompt_text = (image_prompt | llm).invoke({"logo_description": logo}).strip()

    return {
        "name": name,
        "slogan": slogan,
        "concept": concept,
        "logo_description": logo,
        "image_prompt": image_prompt_text
    }

In [ ]:
# Run
result = full_pipeline("Brazilian")
for key, value in result.items():
    print(f"\n🔹 {key.upper()}:\n{value}")